In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
sample_rate = 1.25e9

null_pulse_length = 40e-9
delay_length = 1e-6
pi_pulse_length = 200e-9
laser_readout_length = 300e-9
cw_mw_frequency = 2.8e9

freq_list = np.linspace(2.7e9, 2.9e9, 100)
norm_freq_list = (cw_mw_frequency - freq_list) / sample_rate

freq_rep = 100

In [7]:
%%timeit

single_pulse_length = laser_readout_length + delay_length + pi_pulse_length + 2 * null_pulse_length
single_pulse_samples = int(single_pulse_length * sample_rate)

single_freq_pulse_length = null_pulse_length + single_pulse_length * freq_rep
single_freq_pulse_samples = int(single_freq_pulse_length * sample_rate)

single_sweep_pulse_samples = int(single_freq_pulse_samples * len(freq_list))

delay_pulse_samples = int(np.floor(sample_rate * delay_length))
switch_pulse_samples = int(np.floor(sample_rate * pi_pulse_length))
null_pulse_samples = int(np.floor(null_pulse_length * sample_rate))
laser_readout_pulse_samples = int(np.floor(sample_rate * laser_readout_length))


laser_sequence = np.zeros(single_sweep_pulse_samples)
readout_sequence = np.zeros(single_sweep_pulse_samples)
switch_sequence = np.ones(single_sweep_pulse_samples)

i_sequence = np.zeros(single_sweep_pulse_samples)
q_sequence = np.zeros(single_sweep_pulse_samples)


current_freq_idx = 0
for norm_freq in norm_freq_list:
    end_freq_idx = current_freq_idx + single_freq_pulse_samples
    i_sequence[current_freq_idx:end_freq_idx] = np.sin(2 * np.pi * norm_freq * np.arange(single_freq_pulse_samples))
    q_sequence[current_freq_idx:end_freq_idx] = np.sin(2 * np.pi * norm_freq * np.arange(single_freq_pulse_samples) - np.pi/2)

    for _ in range(freq_rep):
        single_freq_start = current_freq_idx + null_pulse_samples * 2
        
        laser_start = single_freq_start
        laser_stop = laser_start + laser_readout_pulse_samples
        
        switch_start = single_freq_start + delay_pulse_samples
        switch_stop = switch_start + switch_pulse_samples
        
        laser_sequence[laser_start:laser_stop] = np.ones(laser_stop - laser_start)
        readout_sequence[laser_start:laser_stop] = np.ones(laser_stop - laser_start)
        switch_sequence[switch_start:switch_stop] = np.zeros(switch_stop - switch_start)
        
        current_freq_idx += single_pulse_samples

758 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
single_sweep_pulse_samples

NameError: name 'single_sweep_pulse_samples' is not defined

In [ ]:
%matplotlib notebook
fig, (ax, ax2, ax3, ax4, ax5) = plt.subplots(nrows=5, sharex=True)

ax.plot(i_sequence)
ax2.plot(q_sequence)
ax3.plot(laser_sequence)
ax4.plot(switch_sequence)
ax5.plot(readout_sequence)

In [ ]:
int(np.floor(delay_length * sample_rate))